# 데이터 크롤링 03 - 정규식, JSON, 한글 인코딩
* 네이버 실시간 급상승 - 정규식
* 네이버 실시간 검색어 순위 - JSON
* 전화 번호부 - 한글 인코딩


<img width="300" src="http://i.imgur.com/grQHNKG.jpg">

#### 2017 FinanceData http://fb.com/financedata

# 네이버 실시간 급상승 크롤링 - 정규식
* requests.get()
* 파이썬 정규 표현식(regular expression)

http://naver.com
<img src="http://i.imgur.com/AJH8h1d.png" >

짧으면서 최대한 많은 의미를 담기 위한 Python 크롤링 코드. import 문을 제외하면 딱 1줄 !

In [1]:
import re
import requests

html_text = requests.get('http://naver.com').text
re.findall('<span class="ah_k">(.*?)</span>', html_text)[:20]

['하지',
 '트랜스포머',
 '권순호',
 '신연희',
 '최호식',
 '소피아노',
 '디졸브',
 '티아라',
 '에이미',
 '국립자연휴양림',
 '리니지m',
 '병무청',
 '학점계산기',
 '홍은기',
 'bbq',
 '나혼자산다 충재',
 '방탄소년단',
 '민경욱',
 '주말날씨',
 '케네스 배']

# JSON 데이터 - 네이버 실시간 검색어 순위 
실시간 검색어 순위 JOSN 데이터를 반환하는 URL

http://rank.search.naver.com/rank.js

In [2]:
import json, requests
from pandas.io.json import json_normalize

r = requests.get('http://rank.search.naver.com/rank.js')
json_normalize(json.loads(r.text), ['data', 'data'])

,change,cvalue,delta,keyword,rank,ratio,score,tvalue
0,+,683,0,하지,1,.,171,25.21189
1,+,521,0,트랜스포머,2,.,66,16.30625
2,+,168,0,권순호,3,.,336,14.36222
3,+,174,4,신연희,4,+,660,10.51422
4,+,185,1,최호식,5,-,384,9.92218
5,+,214,1,소피아노,6,-,159,9.27699
6,+,78,1,디졸브,7,-,510,6.13401
7,+,94,1,티아라,8,-,54,5.00507
8,+,67,0,에이미,9,.,189,4.65179
9,+,105,1,국립자연휴양림,10,+,75,3.14045


In [3]:
# time stamp

r = requests.get('http://rank.search.naver.com/rank.js')
json.loads(r.text)['ts']

'2017-06-21T14:42:00+0900'

# 한글 인코딩 - 전화번호부 사이트
http://www.isuperpage.co.kr

국내 사이트의 경우 종종 EUC-KR로 인코딩된 데이터만 수용하도록 설계된 경우


지역과 범주를 입력하고 검색하면,
<img src="http://i.imgur.com/jMesQJZ.png">

<!-- TEASER_END -->

다음과 같은 검색 결과를 얻을 수 있다. (특정 지역 + 휴일진료하는 의료기관을 알 수 있다면 좋은 정보가 될 수 도 있을 듯)
<img src="http://i.imgur.com/sI1CjcV.png" >

# euc-kr 인코딩 문제
다른 크롤링 방법과 크게 다르지 않다. 다만, 폼 데이터(requests.post()의 data 인자)로 서버에 전달해야 하는 값이 유니코드가 아니라 "euc-kr"이라는 점이 차이가 있다. (국내 사이트의 상당수가 이런 인코딩 이슈를 안고 있다)

<img src="http://i.imgur.com/V8mi5Ww.png" >

빨간색 테두리 부분을 살펴보자.
FormData 부분을 보면 이상한 문자열로 데이터가 구성되는 것을 볼 수 있는데. 16진수 데이터들이 URL인코딩된 데이터이다.

In [4]:
# searchWord:%C8%DE%C0%CF%C1%F8%B7%E1
# x:37.5640907
# y:126.9979403
# dong:
# city:%BC%AD%BF%EF
# gu:%C1%DF%B1%B8
# addr4:
# addr:%BC%AD%BF%EF+%C1%DF%B1%B8

from urllib.parse import quote, unquote

u = '%C8%DE%C0%CF%C1%F8%B7%E1'
print (unquote(u, encoding='euc-kr'))

u = '%BC%AD%BF%EF'
print (unquote(u, encoding='euc-kr'))

u = '%BC%AD%BF%EF+%C1%DF%B1%B8'
print (unquote(u, encoding='euc-kr'))

휴일진료
서울
서울+중구


# str(유니코드) → bytes(EUC-KR) → URL Quot

In [5]:
b = '휴일진료'.encode('euc-kr')
print( type(b) )
print( quote(b) )

<class 'bytes'>
%C8%DE%C0%CF%C1%F8%B7%E1


# URL Quot (EUC-KR) → str(유니코드)

In [6]:
u = '%C8%DE%C0%CF%C1%F8%B7%E1'
s = unquote(u, encoding='euc-kr')
print(type(s))
print(s)

<class 'str'>
휴일진료


# 해결책
해결책은 무척 간단하다. str(유니코드)를 EUC-KR로 인코딩을 해주면 된다.

```python
str.encode('euc-kr')
```

크롤링과 한글 인코딩에 대한 조금 더 상세한 내용은 아래 포스팅을 참조하자.

* [[FAQ] 크롤링 데이터의 한글이 깨져요](https://financedata.github.io/posts/faq_crawling_data_encoding.html)

In [7]:
import requests

url = 'http://www.isuperpage.co.kr/search.asp'

data = {
    'searchWord': '휴일진료'.encode('euc-kr'),
    'dong':''.encode('euc-kr'),
    'city': '서울'.encode('euc-kr'),
    'gu': '중구'.encode('euc-kr'),
}

r = requests.post(url, data=data)
r.text[:1000]

'\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<title>:: 대한민국 모든 전화번호 검색은 한국전화번호부 ::</title>\r\n<link rel="stylesheet" type=\'text/css\' href="/css3/base_n_n.css"/>\r\n<link href="/css3/dcaccordion.css" rel="stylesheet" type="text/css" />\r\n<link href="/jqy/jquery-ui.css" rel="stylesheet" type="text/css" />\r\n<script type="text/javascript" src="/jqy/jquery-1.11.2.min.js"></script>\r\n<script src="/jqy/jquery-ui-1.10.3.custom.min.js"></script>\r\n<script src="/jqy/jquery.ba-dotimeout.min.js"></script>\r\n<script src="/jqy/common_web.js"></script>\r\n<script src="/jqy/placeholders.min.js"></script>\r\n\r\n  \r\n\r\n\r\n<script language="javascript">\r\n\r\n\r\n$(function() {\r\n\t$("img.lmimg").mouseover(function() {\r\n\t\r\n\t\t$(this).attr("src",$(this).attr("src").replace("off","on"));\r\n\t});\r\n  $("img.lmimg").mouseout(function() {\r\n        $(this).css("display","");\r\n\t\t$(this).at

In [8]:
from bs4 import  BeautifulSoup

soup = BeautifulSoup(r.text, 'lxml')
search_result = soup.find('div', {'id':'search_result'})

In [9]:
lis = search_result.find_all('li')[2:]
for li in lis:
    divs = li.find_all('div')

    # div[0]
    title = divs[0].a.text # 상호
    spans = divs[0].find_all('span')
    search = spans[1].text # 검색어

    # div[1]
    spans = divs[1].find_all('span') 
    phone = spans[0].text # 전화번호
    addr = spans[1].text  # 주소
    print(spans[2])
    print( "%s, %s, %s, %s" % (title, search, phone, addr) )

<span class="load2"><img src="/image/loadi.png"/></span>
중정한의원, 휴일진료병원, 02-3789-7017, 서울 중구 명동1가 7-1
<span class="load2"><img src="/image/loadi.png"/></span>
하버드마취통증의학과의원, 휴일진료병원, 02-2236-8188, 서울 중구 무학동 1
<span class="load2"><img src="/image/loadi.png"/></span>
반도병원, 휴일진료병원, 02-2252-0202, 서울 중구 신당2동 422-4
<span class="load2"><img src="/image/loadi.png"/></span>
유태석내과의원, 휴일진료병원, 02-2233-7076, 서울 중구 신당3동 373-3
<span class="load2"><img src="/image/loadi.png"/></span>
유태석내과의원, 휴일진료병원, 02-2233-7076, 서울 중구 신당3동 373-3
<span class="load2"><img src="/image/loadi.png"/></span>
청구경희한의원, 휴일진료병원, 02-2236-1075, 서울 중구 신당4동 309-8
<span class="load2"><img src="/image/loadi.png"/></span>
약수한의원, 휴일진료병원, 02-2237-3175, 서울 중구 신당4동 370-14
<span class="load2"><img src="/image/loadi.png"/></span>
서울내과의원, 휴일진료병원, 02-2256-7575, 서울 중구 신당6동 294-13
<span class="load2"><img src="/image/loadi.png"/></span>
B N B치과의원, 휴일진료병원, 02-2118-8085, 서울 중구 을지로6가 18-21
<span class="load2"><img src="/image/loadi.png"/></span>
헬로에

----
#### 2017 FinanceData http://fb.com/financedata http://financedata.github.com